In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the btuB gene
model0['lb_list']['EX_cbl1_e ']=0
model0['lb_list']['EX_cbl1_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                                    if i == 0:
                                        # gene upregulation for Fe3+ intake
                                        if 'EX_fe3_e' in rea:
                                            public_reaction_ub_list[i][rea][0, m] = public_reaction_ub_list[i][rea][0, m] * 1.2
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([12, 14,  9, 12,  3,  3, 16,  3,  8,  4, 14, 16, 15,  7,  3,  9,  3,
       19, 15,  1, 16,  9, 14, 10,  1,  1,  8, 15, 19, 16]), 1: array([ 9,  9,  8, 16, 14, 16, 10, 16, 14, 11,  9,  5,  1, 16, 20, 12, 11,
       10, 16,  1, 15, 12, 15,  5, 14,  8, 15,  5,  2,  4])}
1
strain_number:  0 11.333333333333334
strain_various:  0 6.891218244177796
strain_number:  1 10.633333333333333
strain_various:  1 4.996554368317787
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([12, 14,  9, 12,  3,  3, 16,  3,  8,  4, 14, 16, 15,  7,  3,  9,  3,
       19, 15,  1, 16,  9, 14, 10,  1,  1,  8, 15, 19, 16]), 1: array([ 9,  9,  8, 16, 14, 16, 10, 16, 14, 11,  9,  5,  1, 16, 20, 12, 11,
       10, 16,  1, 15, 12, 15,  5, 14,  8, 15,  5,  2,  4])}
_2:  {0: [0.6399142351471028, 0.2857391744693639], 1: [0.6399142351471028, 0.2857391744693639], 2: [0.6399142351471028, 0.2857391744693639], 3: [0.6399142351471028, 0.2857391744693639], 4: [0.6399142351471028, 0.2857391744693639], 

glc__D_e_1:  -8.10477890264607
glc__D_e_1:  -0.9039416501616824
glc__D_e_1:  -11.18217576332701
glc__D_e_1:  -18.07342171266218
glc__D_e_1:  -0.9970510139633646
glc__D_e_1:  -14.562115803722815
glc__D_e_1:  -5.333798198387791
glc__D_e_1:  -20.005982597794247
glc__D_e_1:  -9.621785484764327
glc__D_e_1:  -0.2177976175504437
glc__D_e_1:  -9.588438945922704
glc__D_e_1:  -18.95829495495121
glc__D_e_1:  -1.016121738387918
glc__D_e_1:  -13.99657754013777
glc__D_e_1:  -16.315262016395426
glc__D_e_1:  -0.7733678839935969
glc__D_e_1:  -30.301540743442754
glc__D_e_1:  -8.133696106790657
glc__D_e_1:  -12.56077534465402
glc__D_e_1:  -21.330694746898356
glc__D_e_1:  -0.5166968671730565
glc__D_e_1:  -32.982879238888884
glc__D_e_1:  -13.931318012871909
glc__D_e_1:  -42.05096182643729
glc__D_e_1:  -19.141175595574293
glc__D_e_1:  -13.478910753741012
glc__D_e_1:  -27.302129826261222
glc__D_e_1:  -9.033024725561464
strain_number:  0 38.1
strain_various:  0 15.373895624293365
strain_number:  1 1.7
strain_

glc__D_e_1:  -1.7226074818736088
glc__D_e_1:  -4.2445969543957744
glc__D_e_1:  -1.4735550766303451
glc__D_e_1:  -3.635097304242259
glc__D_e_1:  -2.770830053160669
glc__D_e_1:  -0.6219610061380525
glc__D_e_1:  -2.977466755833391
glc__D_e_1:  -1.5925062868202144
glc__D_e_1:  -3.2109905791442763
glc__D_e_1:  -1.5519834035218973
glc__D_e_1:  -0.8823438263238436
glc__D_e_1:  -1.8041211041809981
glc__D_e_1:  -1.9576183004917742
glc__D_e_1:  -2.44314887160812
glc__D_e_1:  -2.7233351334895097
glc__D_e_1:  -1.933630769938967
glc__D_e_1:  -3.3722934582824458
glc__D_e_1:  -3.2070690136688293
glc__D_e_1:  -2.201611443548809
glc__D_e_1:  -0.7304492509600365
glc__D_e_1:  -1.803982174981594
glc__D_e_1:  -0.56240568466031
glc__D_e_1:  -0.2601030485086233
strain_number:  0 3.566666666666667
strain_various:  0 1.9779338265529063
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 864.8158891547048, 363.6958506534745, 399.08562673885257, 382.6509348108033, 394.12364609

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 18,  4,  7, 19,  8, 12, 15, 14,  5, 18, 11, 14, 15,  4, 16, 18,
       16, 19, 16, 10, 17,  1, 14,  2,  6, 20, 18, 14,  8]), 1: array([18, 10, 12,  4,  2,  6, 13, 16, 20,  2,  8, 17, 17, 15, 18,  3,  4,
       10, 18,  9, 14,  8,  7, 17, 10,  3, 15, 16,  6, 15])}
1
strain_number:  0 14.033333333333333
strain_various:  0 6.882263839432165
strain_number:  1 11.1
strain_various:  1 5.5578772926361015
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 18,  4,  7, 19,  8, 12, 15, 14,  5, 18, 11, 14, 15,  4, 16, 18,
       16, 19, 16, 10, 17,  1, 14,  2,  6, 20, 18, 14,  8]), 1: array([18, 10, 12,  4,  2,  6, 13, 16, 20,  2,  8, 17, 17, 15, 18,  3,  4,
       10, 18,  9, 14,  8,  7, 17, 10,  3, 15, 16,  6, 15])}
_2:  {0: [0.6399142351471028, 0.2857391744693639], 1: [0.6399142351471028, 0.2857391744693639], 2: [0.6399142351471028, 0.2857391744693639], 3: [0.6399142351471028, 0.2857391744693639], 4: [0.6399142351471028, 0.2857391744693639], 5: [0.6399142

glc__D_e_1:  -0.5263223870388343
glc__D_e_1:  -20.237131390614962
strain_number:  0 43.833333333333336
strain_various:  0 16.039707672592485
strain_number:  1 2.2333333333333334
strain_various:  1 0.9893881386437221
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 697.6697412494625, 348.1578830526199, 370.7561364074522, 450.72715424195695, 530.6181762681763, 487.62782859187917, 427.2703365249287, 420.9838636211394, 464.76021628631014, 396.70386553168606, 357.37750777840404, 315.2797065207528]
slip_r:  315.2797065207528
14
glc__D_e_1:  -8.399649577210191
glc__D_e_1:  -0.5301382762932931
glc__D_e_1:  -25.655108081513298
glc__D_e_1:  -21.867172695486122
glc__D_e_1:  -15.2952774262639
glc__D_e_1:  -27.800733714801595
glc__D_e_1:  -0.10205459662378735
glc__D_e_1:  -12.749395708641828
glc__D_e_1:  -30.70748000729955
glc__D_e_1:  -0.4894942603603667
glc__D_e_1:  -12.125466642695116
glc__D_e_1:  -27.56701455067646
glc__D_e_1:  -0.41798193633124825
glc__D_e_1:  -17.804262429394683
glc__D_e_1:  -12.28

glc__D_e_1:  -5.028086599750006
glc__D_e_1:  -4.886095304800302
glc__D_e_1:  -4.295620847173569
glc__D_e_1:  -1.6756409103014989
glc__D_e_1:  -3.2225457671154825
glc__D_e_1:  -0.08590542589228772
glc__D_e_1:  -1.641129986212779
glc__D_e_1:  -1.218176268241769
glc__D_e_1:  -0.8392492672310323
glc__D_e_1:  -1.1305125254742039
glc__D_e_1:  -2.4467347185241923
glc__D_e_1:  -0.7714292064429837
glc__D_e_1:  -0.5901563198838675
glc__D_e_1:  -0.8637944380267211
glc__D_e_1:  -0.4999043394895599
glc__D_e_1:  -0.3475029923112636
glc__D_e_1:  -0.3213351579859315
glc__D_e_1:  -0.7780664478943701
glc__D_e_1:  -0.39257633229286715
glc__D_e_1:  -0.05837580405310261
glc__D_e_1:  -1.13979522771805
glc__D_e_1:  -4.604250164556223
glc__D_e_1:  -2.7792270821249296
glc__D_e_1:  -1.5783797313015349
glc__D_e_1:  -1.698094580950953
glc__D_e_1:  -3.1229703980916406
glc__D_e_1:  -1.512820429966835
glc__D_e_1:  -1.0896529256440737
glc__D_e_1:  -1.9704840136749575
strain_number:  0 2.8333333333333335
strain_variou

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8,  4,  8,  1, 12, 18, 19,  7, 15,  2,  4,  6,  9, 18, 10, 17, 17,
        5,  8,  9,  7, 10, 18,  1,  4,  4, 10, 16,  6, 12]), 1: array([ 1,  5, 20,  8, 10,  9, 11, 14, 12, 12, 10,  6, 10, 20,  4,  2,  1,
        2,  6,  6, 18,  2,  2, 18,  3, 12,  2,  2, 15,  4])}
1
strain_number:  0 10.966666666666667
strain_various:  0 6.539537870182843
strain_number:  1 8.233333333333333
strain_various:  1 5.869033613428667
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8,  4,  8,  1, 12, 18, 19,  7, 15,  2,  4,  6,  9, 18, 10, 17, 17,
        5,  8,  9,  7, 10, 18,  1,  4,  4, 10, 16,  6, 12]), 1: array([ 1,  5, 20,  8, 10,  9, 11, 14, 12, 12, 10,  6, 10, 20,  4,  2,  1,
        2,  6,  6, 18,  2,  2, 18,  3, 12,  2,  2, 15,  4])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.285739174469363

glc__D_e_1:  -1.6486206603668379
glc__D_e_1:  -11.701799050946217
glc__D_e_1:  -21.33223656205092
glc__D_e_1:  -10.975486911978894
glc__D_e_1:  -25.003098804778705
glc__D_e_1:  -14.82798893151338
glc__D_e_1:  -14.78907401460958
glc__D_e_1:  -22.5731409342376
glc__D_e_1:  -29.492558994927087
glc__D_e_1:  -33.78163918728862
glc__D_e_1:  -24.423218945633913
glc__D_e_1:  -24.86805506686167
glc__D_e_1:  -26.88704078715653
glc__D_e_1:  -20.101292659749493
glc__D_e_1:  -29.776863936071642
glc__D_e_1:  -26.93095351814914
glc__D_e_1:  -19.810271355381456
glc__D_e_1:  -21.234826485487552
glc__D_e_1:  -7.005437106520922
glc__D_e_1:  -9.170150459565914
glc__D_e_1:  -7.386665600316512
glc__D_e_1:  -1.7703236675606924
glc__D_e_1:  -9.752182701537254
glc__D_e_1:  -15.696500479171743
glc__D_e_1:  -21.586754722466875
glc__D_e_1:  -6.086066117874214
strain_number:  0 45.733333333333334
strain_various:  0 16.540724154508943
strain_number:  1 0.13333333333333333
strain_various:  1 0.33993463423951903
fd_r

glc__D_e_1:  -3.6849508658938217
glc__D_e_1:  -1.7043186787874416
glc__D_e_1:  -3.8056384776466
glc__D_e_1:  -3.0319925950397804
strain_number:  0 4.9
strain_various:  0 2.573583752927682
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 670.027312599681, 352.52903346653346, 324.1162171162171, 405.690000788685, 446.70624879506465, 433.87272319031655, 412.9308644571803, 327.60566500562254, 405.20359494189074, 398.518923586507, 408.0826474054758, 328.52788670698453, 329.43351151930597, 224.85619454810814, 202.73600041623277, 233.5972868485502, 177.14358164663048, 190.52193809983604, 170.18649707780142, 209.35131535131535]
slip_r:  209.35131535131535
22
glc__D_e_1:  -4.129370293404902
glc__D_e_1:  -1.8890083794166013
glc__D_e_1:  -4.971777829497234
glc__D_e_1:  -1.822839849394799
glc__D_e_1:  -2.7146022892577055
glc__D_e_1:  -0.7494176938292502
glc__D_e_1:  -0.7094998241006838
glc__D_e_1:  -0.5933232590782482
glc__D_e_1:  -1.910340698735622
glc__D_e_1

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 10,  6, 19, 15,  9,  5,  4, 15, 19, 18, 17, 16, 16, 20,  2, 20,
        5, 15,  9, 19,  2, 14,  7, 17, 19, 15,  6, 13,  7]), 1: array([ 9, 14,  4, 16,  4,  4, 16,  5, 11,  3, 17,  2, 19, 19, 13, 11,  3,
        3, 11, 18, 13, 15, 15, 13,  5, 10, 10, 18,  2,  3])}
1
strain_number:  0 14.066666666666666
strain_various:  0 7.215415595946101
strain_number:  1 10.2
strain_various:  1 5.74108003776293
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 10,  6, 19, 15,  9,  5,  4, 15, 19, 18, 17, 16, 16, 20,  2, 20,
        5, 15,  9, 19,  2, 14,  7, 17, 19, 15,  6, 13,  7]), 1: array([ 9, 14,  4, 16,  4,  4, 16,  5, 11,  3, 17,  2, 19, 19, 13, 11,  3,
        3, 11, 18, 13, 15, 15, 13,  5, 10, 10, 18,  2,  3])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -0.3263614428431616
glc__D_e_1:  -7.314050417141139
strain_number:  0 47.03333333333333
strain_various:  0 19.57972307147258
strain_number:  1 1.5
strain_various:  1 1.5652475842498528
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 697.6004965751482, 274.11032271392185, 330.2461542843122, 427.62932034182035, 414.5170991831221, 434.0944204204374, 393.7465067407158, 436.853083950638, 426.6248139816416, 419.11746462129827, 343.35191843372195, 403.88263377473436]
slip_r:  403.88263377473436
14
glc__D_e_1:  -0.5561243094926454
glc__D_e_1:  -10.209062769637253
glc__D_e_1:  -14.875307980520816
glc__D_e_1:  -19.44732367591959
glc__D_e_1:  -27.831338322953435
glc__D_e_1:  -21.318771919507615
glc__D_e_1:  -6.8003494746447
glc__D_e_1:  -18.403887191296132
glc__D_e_1:  -5.49424051994596
glc__D_e_1:  -17.572592114707614
glc__D_e_1:  -3.294370066063982
glc__D_e_1:  -22.834561294814723
glc__D_e_1:  -9.864059543534808
glc__D_e_1:  -23.477903789136686
glc__D_e_1:  -0.48970449191180165
glc__D_e

glc__D_e_1:  -5.770080404595887
glc__D_e_1:  -2.295531516258585
glc__D_e_1:  -4.757356931551877
glc__D_e_1:  -4.796557746447348
glc__D_e_1:  -3.1760189457297283
glc__D_e_1:  -2.383775317641406
glc__D_e_1:  -0.9997899402229906
glc__D_e_1:  -0.6602391021077796
glc__D_e_1:  -0.3993560429729659
glc__D_e_1:  -0.34976386196631626
glc__D_e_1:  -0.15248790323024164
glc__D_e_1:  -0.8399175007270892
glc__D_e_1:  -0.9042024034076739
glc__D_e_1:  -1.1168728741031941
glc__D_e_1:  -0.33689944825308793
glc__D_e_1:  -1.6165941141801285
glc__D_e_1:  -1.658394174353087
glc__D_e_1:  -0.8607120384414995
glc__D_e_1:  -0.4103549095305987
glc__D_e_1:  -0.3244320109490082
glc__D_e_1:  -1.9254747115022726
glc__D_e_1:  -2.282569784430059
glc__D_e_1:  -1.8630154612041958
glc__D_e_1:  -1.5071334081872645
glc__D_e_1:  -3.16091566177901
glc__D_e_1:  -3.225914613057576
glc__D_e_1:  -3.888398783196009
strain_number:  0 3.566666666666667
strain_various:  0 2.538809869910615
strain_number:  1 0.0
strain_various:  1 0.0

In [10]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([18, 18,  7,  3,  5, 14, 16,  8, 14,  6,  2, 10,  7,  1, 11, 10, 13,
        9, 11,  1,  4, 19, 20, 20, 16,  9, 14, 13,  8, 19]), 1: array([10, 12, 17, 15,  5, 19, 15, 19,  6,  4, 20,  3, 12,  4,  5, 19, 18,
        8, 16, 10, 14,  6,  4,  8,  8, 14,  2, 19,  2,  9])}
1
strain_number:  0 12.6
strain_various:  0 6.858571279792898
strain_number:  1 10.766666666666667
strain_various:  1 5.846271366340163
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([18, 18,  7,  3,  5, 14, 16,  8, 14,  6,  2, 10,  7,  1, 11, 10, 13,
        9, 11,  1,  4, 19, 20, 20, 16,  9, 14, 13,  8, 19]), 1: array([10, 12, 17, 15,  5, 19, 15, 19,  6,  4, 20,  3, 12,  4,  5, 19, 18,
        8, 16, 10, 14,  6,  4,  8,  8, 14,  2, 19,  2,  9])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -0.25780740003723324
glc__D_e_1:  -15.832035696481242
glc__D_e_1:  -12.160135030603044
glc__D_e_1:  -15.548467903671936
glc__D_e_1:  -25.780832312571825
glc__D_e_1:  -16.97117612650537
glc__D_e_1:  -16.033977544216963
glc__D_e_1:  -0.008315467811756516
glc__D_e_1:  -12.752107666969394
strain_number:  0 41.06666666666667
strain_various:  0 16.02484182622579
strain_number:  1 1.5333333333333334
strain_various:  1 1.4772346537440224
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 711.4417338337615, 355.4328796203797, 326.57852148600426, 417.9018481518482, 527.6803844458581, 485.0503223379063, 450.9137666868236, 423.6278740415433, 456.8493170509389, 424.1422602857758, 412.12384286587644, 420.2568151441118, 360.60324041954635]
slip_r:  360.60324041954635
15
glc__D_e_1:  -15.386532918796698
glc__D_e_1:  -1.1545888118342984
glc__D_e_1:  -8.694913889685743
glc__D_e_1:  -14.938082671374268
glc__D_e_1:  -1.0719636469330975
glc__D_e_1:  -15.971484346671108
glc__D_e_1:  -20.95786425692146


glc__D_e_1:  -0.041800812837793466
glc__D_e_1:  -0.13749979542157464
glc__D_e_1:  -1.1590457708889568
glc__D_e_1:  -1.2901954892471277
glc__D_e_1:  -1.608414259006022
glc__D_e_1:  -3.386697236059886
glc__D_e_1:  -3.104701507264668
glc__D_e_1:  -3.864565977628584
glc__D_e_1:  -1.0279946627679424
glc__D_e_1:  -5.6376017857462815
glc__D_e_1:  -5.470314300503935
glc__D_e_1:  -5.026807452635893
glc__D_e_1:  -6.297919954866707
glc__D_e_1:  -3.6187068565179312
glc__D_e_1:  -4.647065506824868
glc__D_e_1:  -0.9022037388133003
glc__D_e_1:  -2.7638399416020882
glc__D_e_1:  -1.9187693500591707
glc__D_e_1:  -0.8875340369105567
glc__D_e_1:  -0.6850077804893244
glc__D_e_1:  -0.3699597328777964
strain_number:  0 3.5
strain_various:  0 2.753785273643051
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 711.4417338337615, 355.4328796203797, 326.57852148600426, 417.9018481518482, 527.6803844458581, 485.0503223379063, 450.9137666868236, 423.6278740415433, 456.84931705

In [11]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10,  4, 10, 13,  7, 11, 13, 10,  6, 17, 19, 11,  5,  1, 13, 18, 13,
       14, 18,  6,  1,  9,  1, 17, 12, 12,  4, 19, 15,  1]), 1: array([ 6,  7, 13,  1, 18, 13,  6, 19,  2, 12, 13,  8,  4,  7,  5, 19, 20,
        4, 11, 12, 15,  3,  3, 10,  8, 10, 12,  9,  9, 11])}
1
strain_number:  0 12.0
strain_various:  0 6.5979794887020775
strain_number:  1 9.666666666666666
strain_various:  1 5.127269145353001
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10,  4, 10, 13,  7, 11, 13, 10,  6, 17, 19, 11,  5,  1, 13, 18, 13,
       14, 18,  6,  1,  9,  1, 17, 12, 12,  4, 19, 15,  1]), 1: array([ 6,  7, 13,  1, 18, 13,  6, 19,  2, 12, 13,  8,  4,  7,  5, 19, 20,
        4, 11, 12, 15,  3,  3, 10,  8, 10, 12,  9,  9, 11])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -11.324561554552902
glc__D_e_1:  -28.90554235986561
glc__D_e_1:  -22.684460555488503
glc__D_e_1:  -21.235847315466906
glc__D_e_1:  -10.512298715960467
glc__D_e_1:  -45.97492341569564
glc__D_e_1:  -20.370913364530892
glc__D_e_1:  -15.501093158535882
glc__D_e_1:  -14.642168182981742
glc__D_e_1:  -41.631084346544554
glc__D_e_1:  -13.735929251210928
glc__D_e_1:  -15.252759647921067
glc__D_e_1:  -22.328878588665965
glc__D_e_1:  -6.247651441501361
glc__D_e_1:  -12.254833858508896
glc__D_e_1:  -6.69838439445256
glc__D_e_1:  -5.906545356344399
glc__D_e_1:  -14.68702283082671
glc__D_e_1:  -9.85904290765344
glc__D_e_1:  -22.429839437177193
glc__D_e_1:  -27.184776693728292
glc__D_e_1:  -15.075914450240385
strain_number:  0 41.96666666666667
strain_various:  0 18.972758248487633
strain_number:  1 0.13333333333333333
strain_various:  1 0.33993463423951903
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 919.3721815465237, 331.0986699703805, 354.7063879103353, 405.5934044026149, 413.762388926

glc__D_e_1:  -1.4096334070834489
glc__D_e_1:  -1.3788311955111179
glc__D_e_1:  -1.0534478456293934
glc__D_e_1:  -0.9919724781844439
glc__D_e_1:  -0.01759907770110858
glc__D_e_1:  -0.6153514924520933
glc__D_e_1:  -0.038678057728507964
glc__D_e_1:  -0.027944520855486654
glc__D_e_1:  -0.4961474595056543
glc__D_e_1:  -0.8213660109530921
glc__D_e_1:  -0.6869077409099085
glc__D_e_1:  -0.21412099714565413
glc__D_e_1:  -0.6971140303023723
glc__D_e_1:  -0.5644108898626683
glc__D_e_1:  -1.2504102643777646
glc__D_e_1:  -1.7168089989692552
glc__D_e_1:  -1.9566007208942269
glc__D_e_1:  -3.413932940140967
glc__D_e_1:  -1.6378188468828039
glc__D_e_1:  -1.8331119539819476
glc__D_e_1:  -2.2382413275476085
glc__D_e_1:  -1.7082525166881877
glc__D_e_1:  -3.0470001022497253
glc__D_e_1:  -1.3012931110922208
glc__D_e_1:  -1.295467877832379
strain_number:  0 2.433333333333333
strain_various:  0 1.2827920936595902
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 919.37218

In [6]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8, 14, 19,  9,  1,  7,  5,  2,  1, 16,  2, 18,  8, 20,  6, 14,  3,
        9,  2, 15,  5,  7, 18, 19,  5,  5,  8,  5, 13, 10]), 1: array([ 2, 18, 12,  9, 10, 19,  5, 20,  9, 15,  2, 14,  8,  9,  4, 16, 19,
        7, 15, 11,  1,  5, 10, 18, 19, 17,  6, 18,  9,  7])}
1
strain_number:  0 10.566666666666666
strain_various:  0 6.9889595474258925
strain_number:  1 11.133333333333333
strain_various:  1 5.743015081141807
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 8, 14, 19,  9,  1,  7,  5,  2,  1, 16,  2, 18,  8, 20,  6, 14,  3,
        9,  2, 15,  5,  7, 18, 19,  5,  5,  8,  5, 13, 10]), 1: array([ 2, 18, 12,  9, 10, 19,  5, 20,  9, 15,  2, 14,  8,  9,  4, 16, 19,
        7, 15, 11,  1,  5, 10, 18, 19, 17,  6, 18,  9,  7])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693

glc__D_e_1:  -23.67439358159492
glc__D_e_1:  -1.3548683328025461
glc__D_e_1:  -18.47420681785924
glc__D_e_1:  -19.2740964876128
glc__D_e_1:  -1.5156113752122806
glc__D_e_1:  -4.288706611801359
glc__D_e_1:  -12.354515812934956
glc__D_e_1:  -1.3460915974894263
glc__D_e_1:  -0.8772984803542876
glc__D_e_1:  -0.3798429736356468
glc__D_e_1:  -0.8655184068025596
glc__D_e_1:  -4.677835316212455
glc__D_e_1:  -8.528060996513075
glc__D_e_1:  -0.7433288205943818
glc__D_e_1:  -5.949612313794904
glc__D_e_1:  -23.73004935338772
glc__D_e_1:  -1.4105241045953463
glc__D_e_1:  -18.60025316171781
strain_number:  0 44.46666666666667
strain_various:  0 15.897029771487363
strain_number:  1 1.9333333333333333
strain_various:  1 1.4360439485692011
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 822.6048418171097, 309.32410230378497, 425.62240500727336, 425.89116314542025, 414.8174062779326, 473.05058121702865, 498.81848359973367, 453.27279172071565, 333.48197564604294, 355.3362812150544, 439.57735329305444, 430.196

glc__D_e_1:  -1.9151349427018958
glc__D_e_1:  -3.90807861731225
glc__D_e_1:  -1.9407520593692593
glc__D_e_1:  -0.840315030672734
glc__D_e_1:  -1.7443667825759848
glc__D_e_1:  -2.1820076903440215
glc__D_e_1:  -1.428958449066159
glc__D_e_1:  -2.4125908300496475
glc__D_e_1:  -2.771158765121111
glc__D_e_1:  -2.6088756894361644
glc__D_e_1:  -4.977780907809793
glc__D_e_1:  -1.668116109712869
glc__D_e_1:  -6.468023126293957
glc__D_e_1:  -3.588340193650523
glc__D_e_1:  -4.454247160313656
glc__D_e_1:  -3.310233191252308
glc__D_e_1:  -5.3086931564186335
glc__D_e_1:  -0.8029253751860953
glc__D_e_1:  -1.0160354135877907
glc__D_e_1:  -2.117613931918875
strain_number:  0 5.2
strain_various:  0 3.841874542459709
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 822.6048418171097, 309.32410230378497, 425.62240500727336, 425.89116314542025, 414.8174062779326, 473.05058121702865, 498.81848359973367, 453.27279172071565, 333.48197564604294, 355.3362812150544, 439.5773

In [7]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 4,  9,  6,  1, 19, 12,  8, 19, 13, 10, 16,  8, 13, 13, 10, 11, 10,
        8,  3, 17, 20, 16,  7,  3,  1,  9,  4,  8,  9, 14]), 1: array([11, 13, 14, 20, 12,  7, 17, 17,  6,  3, 19,  9, 14,  5, 15,  8, 14,
        3,  3, 10, 20,  8,  6,  4, 12,  7, 10, 10, 19, 11])}
1
strain_number:  0 11.566666666666666
strain_various:  0 6.243307527976568
strain_number:  1 10.9
strain_various:  1 5.146843692983108
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 4,  9,  6,  1, 19, 12,  8, 19, 13, 10, 16,  8, 13, 13, 10, 11, 10,
        8,  3, 17, 20, 16,  7,  3,  1,  9,  4,  8,  9, 14]), 1: array([11, 13, 14, 20, 12,  7, 17, 17,  6,  3, 19,  9, 14,  5, 15,  8, 14,
        3,  3, 10, 20,  8,  6,  4, 12,  7, 10, 10, 19, 11])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

strain_number:  0 43.7
strain_various:  0 17.733489974245526
strain_number:  1 1.6666666666666667
strain_various:  1 2.0221001184137464
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 540.6546425710512, 321.80118894263626, 389.45062263005804, 456.18718058257525, 455.7020218886942, 461.0701682734191, 473.1540599180415, 448.6004780007365, 462.54148475193267, 461.26133727659874, 429.4786945867533, 353.27461767213305, 363.60384699717224]
slip_r:  363.60384699717224
15
glc__D_e_1:  -6.078522605467455
glc__D_e_1:  -4.494855480706955
glc__D_e_1:  -10.24520700230481
glc__D_e_1:  -15.47631433446908
glc__D_e_1:  -0.08984859789729072
glc__D_e_1:  -28.783707055412446
glc__D_e_1:  -12.773560101341163
glc__D_e_1:  -8.151536551272624
glc__D_e_1:  -23.47424895909367
glc__D_e_1:  -7.880611251599167
glc__D_e_1:  -15.421062535103045
glc__D_e_1:  -4.035519022611064
glc__D_e_1:  -11.441219014613823
glc__D_e_1:  -4.272025208739729
glc__D_e_1:  -19.86073213883394
glc__D_e_1:  -11.45209520470302
glc__D_e_1:  -26.4

glc__D_e_1:  -0.2122538306482462
glc__D_e_1:  -0.4303675825995248
glc__D_e_1:  -0.023673480977245864
glc__D_e_1:  -0.5246436154996617
glc__D_e_1:  -0.2924489090313033
glc__D_e_1:  -1.1312257768044387
glc__D_e_1:  -1.3887472919907957
glc__D_e_1:  -2.5636755666460735
glc__D_e_1:  -3.1231813373062915
glc__D_e_1:  -2.6378747688018596
glc__D_e_1:  -2.5395204780719958
glc__D_e_1:  -6.528145048890192
glc__D_e_1:  -4.358021467218947
glc__D_e_1:  -9.917258561316565
glc__D_e_1:  -5.204540256215062
glc__D_e_1:  -7.730016923760696
glc__D_e_1:  -10.24611353967099
glc__D_e_1:  -9.901758278307911
glc__D_e_1:  -4.986383082647187
strain_number:  0 5.1
strain_various:  0 4.3
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 540.6546425710512, 321.80118894263626, 389.45062263005804, 456.18718058257525, 455.7020218886942, 461.0701682734191, 473.1540599180415, 448.6004780007365, 462.54148475193267, 461.26133727659874, 429.4786945867533, 353.27461767213305, 363.60384699

In [8]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 4, 19,  2, 10, 19,  2, 20, 18, 19, 16,  7, 15, 12,  6,  3, 17, 20,
       14,  4,  2, 10,  2, 10,  8,  1, 19,  3,  7, 19,  6]), 1: array([ 6,  5, 11,  8, 12,  3,  8, 12, 11, 12,  3,  3,  4, 11, 10, 17,  9,
       19,  6, 11,  2, 15,  1, 13, 16,  2,  9, 15,  9,  7])}
1
strain_number:  0 12.133333333333333
strain_various:  0 8.040453276332677
strain_number:  1 9.0
strain_various:  1 4.760952285695233
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 4, 19,  2, 10, 19,  2, 20, 18, 19, 16,  7, 15, 12,  6,  3, 17, 20,
       14,  4,  2, 10,  2, 10,  8,  1, 19,  3,  7, 19,  6]), 1: array([ 6,  5, 11,  8, 12,  3,  8, 12, 11, 12,  3,  3,  4, 11, 10, 17,  9,
       19,  6, 11,  2, 15,  1, 13, 16,  2,  9, 15,  9,  7])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -24.79291423656179
glc__D_e_1:  -37.21259826981264
glc__D_e_1:  -25.346174761014353
glc__D_e_1:  -19.949813056071886
glc__D_e_1:  -21.132772145326705
glc__D_e_1:  -17.410907498133653
glc__D_e_1:  -10.724768874939585
glc__D_e_1:  -12.57496006626066
glc__D_e_1:  -7.002910348813395
glc__D_e_1:  -16.203812989179074
glc__D_e_1:  -20.57655118237866
glc__D_e_1:  -18.79315733530307
glc__D_e_1:  -10.323392616881167
glc__D_e_1:  -24.264038115417353
glc__D_e_1:  -16.45825251340253
glc__D_e_1:  -21.799403609025017
glc__D_e_1:  -21.487235555112733
strain_number:  0 39.06666666666667
strain_various:  0 18.092232833149392
strain_number:  1 0.5
strain_various:  1 0.7187952884282609
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 877.102783761646, 358.5682190616402, 383.175381635908, 463.9318368035474, 447.05871541975864, 353.61806075220755, 424.28339731044156, 436.60986496440876, 404.7817244764543, 428.4537311850995, 414.93321480392353, 436.17529342746013, 394.3842706448811, 270.0332917054481]

glc__D_e_1:  -0.8091465513633174
glc__D_e_1:  -0.4101416398819875
glc__D_e_1:  -0.6059029005584475
glc__D_e_1:  -0.331794376641967
glc__D_e_1:  -0.3247165292346852
glc__D_e_1:  -0.02839651371300511
glc__D_e_1:  -0.9212953540868833
glc__D_e_1:  -1.7193347907507417
glc__D_e_1:  -1.1139786384343247
glc__D_e_1:  -4.008532004164808
glc__D_e_1:  -2.9406937714915506
glc__D_e_1:  -6.769422500283229
glc__D_e_1:  -8.365975964814705
glc__D_e_1:  -1.6797491956435855
glc__D_e_1:  -5.389312081280469
glc__D_e_1:  -4.1457986602744645
glc__D_e_1:  -4.811251218366997
glc__D_e_1:  -1.6062661201587578
glc__D_e_1:  -2.663090395142097
glc__D_e_1:  -1.7436558950600765
glc__D_e_1:  -3.5028441491092703
strain_number:  0 3.0
strain_various:  0 3.2145502536643185
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 877.102783761646, 358.5682190616402, 383.175381635908, 463.9318368035474, 447.05871541975864, 353.61806075220755, 424.28339731044156, 436.60986496440876, 404.7817244

In [9]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10, 18, 11, 15, 13, 10, 14, 12,  9, 19,  8, 15,  1, 19,  7,  9, 11,
        3, 16,  6,  7,  4,  4, 20, 18, 20,  5,  2, 13, 10]), 1: array([14,  9, 11, 11, 16, 10,  7,  1,  6,  1, 17,  7, 19, 16,  4,  4, 10,
       20, 19, 17,  2, 11,  3,  4, 14, 18, 10, 11,  3, 18])}
1
strain_number:  0 12.766666666666667
strain_various:  0 6.666749999479173
strain_number:  1 10.433333333333334
strain_various:  1 5.936796741977576
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10, 18, 11, 15, 13, 10, 14, 12,  9, 19,  8, 15,  1, 19,  7,  9, 11,
        3, 16,  6,  7,  4,  4, 20, 18, 20,  5,  2, 13, 10]), 1: array([14,  9, 11, 11, 16, 10,  7,  1,  6,  1, 17,  7, 19, 16,  4,  4, 10,
       20, 19, 17,  2, 11,  3,  4, 14, 18, 10, 11,  3, 18])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -0.551289529997474
glc__D_e_1:  -11.711355450312887
glc__D_e_1:  -0.6288217078433433
glc__D_e_1:  -0.44582309257774555
glc__D_e_1:  -0.8933547439861869
glc__D_e_1:  -5.987245973068283
glc__D_e_1:  -7.962822514902301
glc__D_e_1:  -9.304133442637035
glc__D_e_1:  -16.25680235029049
strain_number:  0 43.56666666666667
strain_various:  0 16.989571965048313
strain_number:  1 1.3
strain_various:  1 1.1298967504452195
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 854.1689876815651, 364.7990161154635, 376.0441291164974, 511.23732831496005, 526.5696587156477, 437.76540627667293, 423.00749897247584, 392.5840360875232, 405.539748841864, 430.31598127343653, 399.82716163062287, 383.19642696191136, 376.3844789771437]
slip_r:  376.3844789771437
15
glc__D_e_1:  -9.604594942935925
glc__D_e_1:  -6.583369924713207
glc__D_e_1:  -7.522770176333686
glc__D_e_1:  -11.53679090573661
glc__D_e_1:  -13.382164131951503
glc__D_e_1:  -11.84317792007441
glc__D_e_1:  -32.294482178366636
glc__D_e_1:  -8.687839

glc__D_e_1:  -0.10117576600526756
glc__D_e_1:  -0.12114324905208762
glc__D_e_1:  -0.26150682609735476
glc__D_e_1:  -0.08409299929947345
glc__D_e_1:  -1.6093456992432689
glc__D_e_1:  -1.639745325570264
glc__D_e_1:  -2.6469155932719355
glc__D_e_1:  -3.3830640305260276
glc__D_e_1:  -1.8638496418158708
glc__D_e_1:  -3.190788580272355
glc__D_e_1:  -2.7040212326220354
glc__D_e_1:  -9.679001777077591
glc__D_e_1:  -3.4757109890681406
glc__D_e_1:  -3.3092083604928444
glc__D_e_1:  -2.8817712589538202
glc__D_e_1:  -4.469224436889959
glc__D_e_1:  -3.832255764698321
glc__D_e_1:  -0.3817126167014746
glc__D_e_1:  -1.7380250847936334
glc__D_e_1:  -1.4370920216567873
glc__D_e_1:  -2.0606293581033013
glc__D_e_1:  -0.41174291022490933
glc__D_e_1:  -1.7014404418380904
strain_number:  0 3.1666666666666665
strain_various:  0 2.745703714694812
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 854.1689876815651, 364.7990161154635, 376.0441291164974, 511.23732831496005, 52

In [10]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2, 10, 14, 15,  9, 19, 11,  5, 12, 12, 17, 13, 17, 18, 15,  1,  3,
       19,  7,  2, 19,  8, 14,  9, 14,  4,  6,  1, 15,  2]), 1: array([ 5, 17,  1,  7, 18, 16, 11,  4, 10,  4,  2, 10, 18, 18,  2, 17, 10,
        2, 18,  1, 17,  7,  2,  3, 11, 15, 15, 13, 15, 18])}
1
strain_number:  0 12.066666666666666
strain_various:  0 6.985381561200187
strain_number:  1 10.233333333333333
strain_various:  1 6.243307527976569
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2, 10, 14, 15,  9, 19, 11,  5, 12, 12, 17, 13, 17, 18, 15,  1,  3,
       19,  7,  2, 19,  8, 14,  9, 14,  4,  6,  1, 15,  2]), 1: array([ 5, 17,  1,  7, 18, 16, 11,  4, 10,  4,  2, 10, 18, 18,  2, 17, 10,
        2, 18,  1, 17,  7,  2,  3, 11, 15, 15, 13, 15, 18])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -23.258608138322778
glc__D_e_1:  -38.49110244772268
glc__D_e_1:  -30.13991030401337
glc__D_e_1:  -23.3798299331267
glc__D_e_1:  -28.683903980896147
glc__D_e_1:  -8.802343666581013
glc__D_e_1:  -20.56014196526149
glc__D_e_1:  -9.23132429451314
glc__D_e_1:  -14.663827237828288
glc__D_e_1:  -6.573494719992718
glc__D_e_1:  -21.803779434271007
glc__D_e_1:  -4.358051809120119
glc__D_e_1:  -18.067454851042324
glc__D_e_1:  -8.438888470332527
glc__D_e_1:  -22.860008413901024
glc__D_e_1:  -21.822017413855463
glc__D_e_1:  -26.856805626216374
glc__D_e_1:  -0.15923885629774048
glc__D_e_1:  -30.024165819246576
glc__D_e_1:  -24.276486378966883
glc__D_e_1:  -19.606692132656914
glc__D_e_1:  -0.1289026543176952
glc__D_e_1:  -9.579253833802156
strain_number:  0 36.733333333333334
strain_various:  0 18.388281292412536
strain_number:  1 1.1666666666666667
strain_various:  1 1.8454147380888546
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 894.8394497659203, 403.56533378901804, 368.25614663114675, 

glc__D_e_1:  -3.494804243450387
glc__D_e_1:  -5.293189091465103
glc__D_e_1:  -4.283580741382845
glc__D_e_1:  -4.8459914131468045
glc__D_e_1:  -9.259909168308887
glc__D_e_1:  -5.665250300874582
strain_number:  0 4.2
strain_various:  0 3.1453669632227865
strain_number:  1 0.06666666666666667
strain_various:  1 0.24944382578492943
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 894.8394497659203, 403.56533378901804, 368.25614663114675, 395.06205852014426, 417.4578586618061, 461.1225171886937, 454.37137667145356, 400.9983786506161, 400.9856129894293, 368.4190967971407, 377.86940612767586, 410.71356825571985, 289.1337849297982, 347.43595666210035, 270.76679370278146, 258.063515997409, 266.8004175051222, 198.92733498580273, 234.34261901781082]
slip_r:  234.34261901781082
21
glc__D_e_1:  -0.04704733981195908
glc__D_e_1:  -0.1979342808897422
glc__D_e_1:  -1.0789660371829637
glc__D_e_1:  -1.010926685400662
glc__D_e_1:  -0.27184963808868556
glc__D_e_1:  -0.4158109289791737
glc__D_e_1:  -0.34183674260